In [1]:
from datasets import load_from_disk, concatenate_datasets
from transformers import MimiModel, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")
model = MimiModel.from_pretrained("kyutai/mimi")
model = model.to("cuda")

/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/transformers/models/mimi/modeling_mimi.py:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


NOTE: dataset creation script is elsewhere.

If you are Jacob Keisling, please create a file `.env` in this repo and put your HuggingFace token under the `HUGGINGFACE_TOKEN=sk-xxxx` variable. Since you are probably not Jacob Keisling, if you want to re-upload, please pick a repo you actually own.

In [ ]:
from datasets import load_from_disk
from dotenv import load_dotenv
import os
load_dotenv()

# If you created this from scratch
dataset = load_from_disk("./encoded_libritts")
dataset = dataset.with_format("torch")
dataset = dataset.rename_column(original_column_name="sentences", new_column_name="text_normalized")
# Uncomment if this is your first time pushing your new dataset
dataset.push_to_hub("jkeisling/libritts-r-mimi", token=os.getenv("HUGGINGFACE_TOKEN"))

NameError: name 'load_from_disk' is not defined

In [3]:
import torch
import torchaudio

codes = dataset['dev.clean'][0]['codes']
codes

tensor([[1049, 1114, 1609,  784,  499,  260, 1011,    8, 1407,  540, 1615,  561,
         1945,  201, 1324,  668,  376, 1849,    9, 1921, 1921, 1683,  228,  897,
         1677,  518],
        [ 811, 1149,  739,  410, 1367, 1305, 2046, 1287,  886, 1995, 1727,  678,
         1455,  352, 1914, 1504, 1138, 1154,  669, 1217, 1450, 1003, 1711,  488,
          342,  844],
        [ 373, 1464, 2013, 1306,  102,  561,  852,  267,  442,  718, 1501, 1455,
          233, 1015,  963,   29,  496, 1728,  783, 1870,  879, 1802, 1523,  231,
          333,  199],
        [ 131, 1957,   58,  500, 1489, 1451, 1946, 1800,   68,   46,  214, 2008,
          539,  217,  403, 1270,  112, 1922,  104,  962, 1194, 1785,   36,  603,
          212,  104],
        [  68,  674, 1446, 1356,  457,   49,  541,  505,  122,  457, 1773,   44,
          823, 1423, 1693,  644,  297,  272,  720,   91,  635, 1187,   64,  485,
          844,  417],
        [1739,  954,  935,  640, 1509,   63,  398, 1824,  707, 1948, 1273, 1832,

In [4]:
codes_input = codes.to('cuda').to(torch.long).unsqueeze(0)
out_pcm = model.decode(codes_input)
torchaudio.save("out.wav", out_pcm.audio_values[0].to("cpu"), 24000)